In [1]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

In [2]:
def load_dataset():
    mask_images = "dataset/with_mask/*"
    no_mask_images = "dataset/without_mask/*"
    image_size = 28
    x_train_list = []
    y_train_list = []
    for image in glob.glob(mask_images):
        try:
            img = cv2.imread(image, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (image_size, image_size))
            x_train_list.append(img)
            y_train_list.append([0])
        except:
            pass

    for image in glob.glob(no_mask_images):
        try:
            img = cv2.imread(image, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (image_size, image_size))
            x_train_list.append(img)
            y_train_list.append([1])
        except:
            pass

    x_train_data = np.asarray(x_train_list)
    y_train_data = np.asarray(y_train_list)
    
    # Normalize the data
    x_train_data = x_train_data / 255
    
    print(x_train_data.shape)
    print(y_train_data.shape)
    
    return x_train_data, y_train_data

In [3]:
X, Y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

(3810, 28, 28, 3)
(3810, 1)


((2857, 28, 28, 3), (2857, 1), (953, 28, 28, 3), (953, 1))

In [4]:
def build_model():
    
    # Define the CNN model
    model = Sequential()

    # first Convolutional Layer
    model.add(Conv2D(32, (5,5), activation='relu', input_shape=(28, 28,3)))
    model.add(MaxPooling2D((2, 2)))

    # second Convolutional Layer
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    # Flatten Layer
    model.add(Flatten())
    
    # first Fully-Connected Layer
    model.add(Dense(600, activation='relu'))
    # first Dropout Layer
    model.add(Dropout(0.5))
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    
    return model

In [5]:
model = build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 600)               615000    
_________________________________________________________________
dropout (Dropout)            (None, 600)               0

In [6]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
history = model.fit(X_train, y_train,
          batch_size=100,
          epochs=20,
          verbose=1)

Epoch 1/20
29/29 [==============================] - 1s 40ms/step - loss: 0.4140 - accuracy: 0.8127
Epoch 2/20
29/29 [==============================] - 1s 39ms/step - loss: 0.2028 - accuracy: 0.9335
Epoch 3/20
29/29 [==============================] - 1s 37ms/step - loss: 0.1676 - accuracy: 0.9408
Epoch 4/20
29/29 [==============================] - 1s 47ms/step - loss: 0.1769 - accuracy: 0.9394
Epoch 5/20
29/29 [==============================] - 2s 54ms/step - loss: 0.1672 - accuracy: 0.9419
Epoch 6/20
29/29 [==============================] - 1s 46ms/step - loss: 0.1296 - accuracy: 0.9573
Epoch 7/20
29/29 [==============================] - 1s 47ms/step - loss: 0.1258 - accuracy: 0.9541
Epoch 8/20
29/29 [==============================] - 1s 37ms/step - loss: 0.0913 - accuracy: 0.9657
Epoch 9/20
29/29 [==============================] - 1s 43ms/step - loss: 0.0845 - accuracy: 0.9699
Epoch 10/20
29/29 [==============================] - 1s 39ms/step - loss: 0.0739 - accuracy: 0.9748
Epoch 11/

In [8]:
# Test the model's accuracy with the test data
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)


30/30 [==============================] - 0s 5ms/step - loss: 0.1681 - accuracy: 0.9612
Test accuracy: 0.961175262928009


In [9]:
# Call the history attribute to store all results per epoch to a DataFrame
hist = pd.DataFrame(history.history)
# Add epoch number to the DataFrame
hist['epoch'] = history.epoch
# View last five epochs
hist.tail()

,loss,accuracy,epoch
15,0.020956,0.99370,15
16,0.026055,0.99020,16
17,0.012083,0.99755,17
18,0.008265,0.99825,18
19,0.010976,0.99685,19
